# VacationPy

Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import json 
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,caleta de carquin,-11.0925,-77.6267,67.46,83,100,9.37,PE,1697339027
1,1,hermanus,-34.4187,19.2345,59.07,76,98,16.67,ZA,1697339027
2,2,ilulissat,69.2167,-51.1000,17.62,79,20,9.22,GL,1697339027
3,3,moya,28.0906,-15.5883,78.67,48,73,5.53,ES,1697339027
4,4,adamstown,-25.0660,-130.1015,70.99,80,96,5.61,PN,1697339027


Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    alpha = 0.5,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
## A max temperature lower than 50 degrees but higher than 21.
## Wind speed less than 7.5 mph.
## Zero cloudiness.

narrow_city_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 50) & 
                                    (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 7.5)]

# Drop any rows with null values
narrow_city_df = narrow_city_df.dropna(how='any')

# Display sample data
narrow_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,the pas,53.8251,-101.2541,39.22,93,0,2.30,CA,1697339031
47,47,thompson,55.7435,-97.8558,41.16,81,0,2.30,CA,1697338959
134,134,island harbour,50.7240,-1.2758,44.02,89,0,3.00,GB,1697339042
185,185,homer,59.6425,-151.5483,43.77,72,0,0.00,US,1697339049
266,266,damavand,35.7178,52.0650,49.78,66,0,2.84,IR,1697339060
271,271,xining,36.6167,101.7667,47.84,34,0,1.88,CN,1697339061
294,294,elin-yurt,43.6731,44.9589,49.01,79,0,5.73,RU,1697339063
303,303,roma,43.2128,-75.4557,48.96,86,0,0.00,US,1697339064
390,390,saint-francois,46.4154,3.9054,42.24,100,0,2.30,FR,1697339076
462,462,colonia,50.9333,6.9500,46.89,90,0,3.44,DE,1697338894


Step 3: Create a new DataFrame called hotel_df.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
35,the pas,CA,39.22,53.8251,-101.2541,
47,thompson,CA,41.16,55.7435,-97.8558,
134,island harbour,GB,44.02,50.7240,-1.2758,
185,homer,US,43.77,59.6425,-151.5483,
266,damavand,IR,49.78,35.7178,52.0650,
271,xining,CN,47.84,36.6167,101.7667,
294,elin-yurt,RU,49.01,43.6731,44.9589,
303,roma,US,48.96,43.2128,-75.4557,
390,saint-francois,FR,42.24,46.4154,3.9054,
462,colonia,DE,46.89,50.9333,6.9500,


Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 10,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
  
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
the pas - nearest hotel: Wescana Inn
thompson - nearest hotel: Thompson Inn
island harbour - nearest hotel: Premier Inn
homer - nearest hotel: Driftwood Inn & Homer Seaside Lodges
damavand - nearest hotel: Avishan Hotel
xining - nearest hotel: Qinghai hotel
elin-yurt - nearest hotel: No hotel found
roma - nearest hotel: Wingate by Wyndham Rome
saint-francois - nearest hotel: Chez Lily
colonia - nearest hotel: Wasserturm Hotel Cologne
adrasmon - nearest hotel: No hotel found


,City,Country,Max Temp,Lat,Lng,Hotel Name
35,the pas,CA,39.22,53.8251,-101.2541,Wescana Inn
47,thompson,CA,41.16,55.7435,-97.8558,Thompson Inn
134,island harbour,GB,44.02,50.7240,-1.2758,Premier Inn
185,homer,US,43.77,59.6425,-151.5483,Driftwood Inn & Homer Seaside Lodges
266,damavand,IR,49.78,35.7178,52.0650,Avishan Hotel
271,xining,CN,47.84,36.6167,101.7667,Qinghai hotel
294,elin-yurt,RU,49.01,43.6731,44.9589,No hotel found
303,roma,US,48.96,43.2128,-75.4557,Wingate by Wyndham Rome
390,saint-francois,FR,42.24,46.4154,3.9054,Chez Lily
462,colonia,DE,46.89,50.9333,6.9500,Wasserturm Hotel Cologne


Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    alpha = 0.5,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["City", "Hotel Name", "Country"]
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)